In [1]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import duckdb
import pandas as pd
import gspread
import re

In [2]:
SCOPES = [
    "https://www.googleapis.com/auth/drive.readonly",
    "https://www.googleapis.com/auth/spreadsheets.readonly",
]
SA_PATH = "../config/poc-pipeline-461721-6aff0d3b47eb.json"  # caminho para o JSON da service account
ROOT_FOLDER_ID = "1umg5he9khM4PP_0rGSt5W9N0J-91DfCo"  # opcionalmente substitua por um ID de pasta específico

creds = service_account.Credentials.from_service_account_file(SA_PATH, scopes=SCOPES)
drive = build("drive", "v3", credentials=creds)
sheets_api = build("sheets", "v4", credentials=creds)
gc = gspread.authorize(creds)

In [17]:
sh = gc.open_by_key("1jww0lMW86Qe743lt8zOf4icWq3_Iz5rvqBbqKwuL9aI")
ws = sh.worksheets()

In [35]:
for ws in sh.worksheets():
    print(ws)

<Worksheet 'inputs' id:704503947>
<Worksheet 'grupo' id:1659275219>
<Worksheet 'receitas' id:1347168356>
<Worksheet 'despesas' id:2002899149>


In [ ]:
def upsert_table_simple_pk(conn, df: pd.DataFrame, table_name: str, key_cols: list):
    """
    Upserts a DataFrame into a DuckDB table using INSERT OR REPLACE.
    Requires the table to have a PRIMARY KEY defined on key_cols.
    """
    if not key_cols:
        raise ValueError("key_cols must be provided for INSERT OR REPLACE upsert.")

    # 1. Criar uma tabela temporária com os novos dados
    temp_table_name = f"{table_name}_temp_new_data"
    df.to_sql(temp_table_name, conn, if_exists="replace", index=False)

    # 2. Verificar se a tabela principal existe e criar se não
    table_exists = conn.execute(f"""
        SELECT COUNT(*) FROM information_schema.tables
        WHERE table_name = '{table_name}'
    """).fetchone()[0] > 0

    if not table_exists:
        # Se a tabela não existe, crie-a a partir dos dados temporários
        conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM {temp_table_name}")
        # Adicione a chave primária
        conn.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({', '.join(key_cols)})")
    else:
        # Se a tabela existe, use INSERT OR REPLACE para upsert
        # Isso funciona porque a tabela principal tem uma PRIMARY KEY
        conn.execute(f"INSERT OR REPLACE INTO {table_name} SELECT * FROM {temp_table_name}")

    # 3. Limpar a tabela temporária
    conn.execute(f"DROP TABLE {temp_table_name}")

In [13]:
import duckdb
import pandas as pd
conn = duckdb.connect(database=":memory:", read_only=False)

In [14]:
# Dados iniciais
data1 = {'id': [1, 2], 'value': ['A', 'B']}
df1 = pd.DataFrame(data1)

In [17]:
upsert_table_simple_pk(conn, df1, "my_data", ["id"])
print("Após primeira inserção:")
print(conn.sql("SELECT * FROM my_data").show())

Após primeira inserção:
┌───────┬─────────┐
│  id   │  value  │
│ int32 │ varchar │
├───────┼─────────┤
│     1 │ A       │
│     2 │ B       │
└───────┴─────────┘

None


C:\Users\davi_\AppData\Local\Temp\ipykernel_17872\1309856605.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(temp_table_name, conn, if_exists="replace", index=False)


In [18]:
data2 = {'id': [1, 3], 'value': ['A_updated', 'C']}
df2 = pd.DataFrame(data2)
upsert_table_simple_pk(conn, df2, 'my_data', ['id'])
print("\nApós upsert:")
print(conn.sql("SELECT * FROM my_data").show())


Após upsert:
┌───────┬───────────┐
│  id   │   value   │
│ int32 │  varchar  │
├───────┼───────────┤
│     1 │ A_updated │
│     2 │ B         │
│     3 │ C         │
└───────┴───────────┘

None


C:\Users\davi_\AppData\Local\Temp\ipykernel_17872\1309856605.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql(temp_table_name, conn, if_exists="replace", index=False)
